In [ ]:
import os
import pandas as pd
import lightgbm as lgb
import optuna
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# 1️⃣ 데이터 로드
train_file = ''
test_file = ''


train_data = TabularDataset(train_file)
test_data = TabularDataset(test_file)

# 2️⃣ 타겟 변수 설정
target_column = "임신 성공 여부"  # 사용자의 데이터에 맞게 수정 필요

if target_column not in train_data.columns:
    raise ValueError(f"'{target_column}' 컬럼이 데이터셋에 없습니다. 올바른 타겟 변수를 설정하세요.")

# 3️⃣ AutoGluon을 이용한 결측값 자동 채우기 (타겟 변수를 고려하여 학습)
predictor = TabularPredictor(label=target_column)  # 타겟 컬럼을 사용하여 결측값 채우기
predictor.fit(train_data)  # 🔹 학습 진행 (결측값 자동 처리 포함)

# 4️⃣ 결측값이 채워진 데이터 변환
processed_train = predictor.transform_features(train_data)  # 학습 데이터 변환
processed_test = predictor.transform_features(test_data)  # 테스트 데이터 변환

# 3️⃣ AutoGluon 모델 저장 (결측치 처리 모델)
autogluon_save_path = ""
predictor.save(autogluon_save_path)
print(f"🚀 AutoGluon 전처리 모델 저장 완료! 저장 경로: {autogluon_save_path}")









Loaded data from: /content/drive/MyDrive/Aimers6/train_3.csv | Columns = 61 / 61 | Rows = 256351 -> 256351
Loaded data from: /content/drive/MyDrive/Aimers6/test_3.csv | Columns = 60 / 60 | Rows = 90067 -> 90067
No path specified. Models will be saved in: "AutogluonModels/ag-20250223_004443"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       9.76 GB / 12.67 GB (77.0%)
Disk Space Avail:   73.37 GB / 107.72 GB (68.1%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundatio

🚀 AutoGluon 전처리 모델 저장 완료! 저장 경로: /content/drive/MyDrive/Aimers6/autogluon_save.csv


In [ ]:
processed_train = predictor.transform_features(train_data)
processed_train[target_column] = train_data[target_column].values

In [ ]:
# 3️⃣ AutoGluon 모델 저장 (결측치 처리 모델)
autogluon_save_path = ""
predictor.save(autogluon_save_path)
print(f"🚀 AutoGluon 전처리 모델 저장 완료! 저장 경로: {autogluon_save_path}")

🚀 AutoGluon 전처리 모델 저장 완료! 저장 경로: /content/drive/MyDrive/Aimers6/autogluon_save.csv


In [ ]:
# 5️⃣ 학습 & 검증 데이터 분할 (80% 학습, 20% 검증)
train_data, val_data = train_test_split(processed_train, test_size=0.2, random_state=42, stratify=processed_train["임신 성공 여부"])

# 6️⃣ LightGBM 학습을 위한 데이터 준비
X_train = train_data.drop(columns=[target_column])
y_train = train_data[target_column]

X_val = val_data.drop(columns=[target_column])
y_val = val_data[target_column]

train_dataset = lgb.Dataset(X_train, label=y_train)
val_dataset = lgb.Dataset(X_val, label=y_val, reference=train_dataset)

In [ ]:
import lightgbm as lgb
print(lgb.__version__)


4.5.0


In [ ]:
# 7️⃣ Optuna를 활용한 하이퍼파라미터 최적화 함수
def objective(trial):
    params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
    "num_leaves": trial.suggest_int("num_leaves", 20, 100),
    "max_depth": trial.suggest_int("max_depth", 3, 12),
    "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
    "subsample": trial.suggest_float("subsample", 0.5, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
    "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
    "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
    "feature_pre_filter": False     # ← 추가된 부분
}


    model = lgb.train(
        params,
        train_dataset,
        valid_sets=[val_dataset],
        num_boost_round=100,
        callbacks=[lgb.early_stopping(10)]
    )
    y_val_pred = model.predict(X_val)
    auc = roc_auc_score(y_val, y_val_pred)
    return auc


# 8️⃣ 최적의 하이퍼파라미터 탐색 (Optuna)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # 20번의 탐색 수행

best_params = study.best_params
print(f"🔍 최적의 하이퍼파라미터: {best_params}")

# 9️⃣ 최적의 하이퍼파라미터로 모델 학습
# Optuna에서 얻은 best_params에 feature_pre_filter 추가
best_params["feature_pre_filter"] = False

model = lgb.train(
    best_params,
    train_dataset,
    valid_sets=[val_dataset],
    num_boost_round=200,
    callbacks=[lgb.early_stopping(20)]
)


# 🔟 LightGBM 모델 저장
model_save_path = ""
model.save_model(model_save_path)
print(f"🚀 LightGBM 모델 저장 완료! 저장 경로: {model_save_path}")

# 1️⃣1️⃣ 검증 데이터 평가
y_val_pred = model.predict(X_val)
auc = roc_auc_score(y_val, y_val_pred)
print(f"검증 데이터 AUC: {auc:.4f}")

[I 2025-02-23 00:39:24,102] A new study created in memory with name: no-name-07791fe2-eedd-49b1-8776-87f32b3aca56


Training until validation scores don't improve for 10 rounds


[I 2025-02-23 00:39:26,329] Trial 0 finished with value: 0.7364812430691244 and parameters: {'learning_rate': 0.14836056560609476, 'num_leaves': 66, 'max_depth': 9, 'min_child_samples': 7, 'subsample': 0.9649765942851571, 'colsample_bytree': 0.7915792482018329, 'reg_alpha': 0.6082937083821309, 'reg_lambda': 0.36846628558851446}. Best is trial 0 with value: 0.7364812430691244.


Early stopping, best iteration is:
[39]	valid_0's auc: 0.736481
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[56]	valid_0's auc: 0.736426


[I 2025-02-23 00:39:32,361] Trial 1 finished with value: 0.7364262914198454 and parameters: {'learning_rate': 0.09713523082338398, 'num_leaves': 100, 'max_depth': 10, 'min_child_samples': 13, 'subsample': 0.5001141442324484, 'colsample_bytree': 0.6688536934557059, 'reg_alpha': 0.8093902598179066, 'reg_lambda': 0.2698113038809846}. Best is trial 0 with value: 0.7364812430691244.


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[46]	valid_0's auc: 0.7369


[I 2025-02-23 00:40:37,063] Trial 18 finished with value: 0.7368997992074915 and parameters: {'learning_rate': 0.11635699625796309, 'num_leaves': 75, 'max_depth': 10, 'min_child_samples': 32, 'subsample': 0.7712513157552451, 'colsample_bytree': 0.604808744774737, 'reg_alpha': 0.7188101723863091, 'reg_lambda': 0.47121410045000417}. Best is trial 9 with value: 0.7371006383726624.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[54]	valid_0's auc: 0.736464


[I 2025-02-23 00:40:41,737] Trial 19 finished with value: 0.7364642322399091 and parameters: {'learning_rate': 0.07245084340894582, 'num_leaves': 90, 'max_depth': 8, 'min_child_samples': 16, 'subsample': 0.8331724758180747, 'colsample_bytree': 0.755953867622671, 'reg_alpha': 0.8959937011914757, 'reg_lambda': 0.15029728409973175}. Best is trial 9 with value: 0.7371006383726624.


🔍 최적의 하이퍼파라미터: {'learning_rate': 0.05207700266245276, 'num_leaves': 79, 'max_depth': 12, 'min_child_samples': 36, 'subsample': 0.6967863403199377, 'colsample_bytree': 0.7537099043819682, 'reg_alpha': 0.8267558705867153, 'reg_lambda': 0.49510737360816537}
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[99]	valid_0's l2: 0.166392
🚀 LightGBM 모델 저장 완료! 저장 경로: /content/drive/MyDrive/model_save
검증 데이터 AUC: 0.7370


In [ ]:
# 1️⃣2️⃣ 테스트 데이터 예측
X_test = processed_test.drop(columns=[target_column], errors="ignore")  # 타겟 컬럼 제거
y_test_pred = model.predict(X_test)

# 1️⃣3️⃣ 예측 결과 저장
submission = pd.DataFrame({target_column: y_test_pred})
submission.to_csv("", index=False)



In [ ]:
# 7️⃣ **최적 모델로 테스트 데이터 예측**
y_test_pred_proba = predictor.predict_proba(test_data)

# 🔹 테스트 데이터 확률값 처리
if isinstance(y_test_pred_proba, pd.DataFrame):
    y_test_pred = y_test_pred_proba.iloc[:, 1]  # DataFrame이면 iloc 사용
else:
    y_test_pred = y_test_pred_proba[:, 1]  # numpy 배열이면 직접 사용

# 8️⃣ **최종 제출 파일 생성**
submission['probability'] = y_test_pred  # 확률값 그대로 저장
submission.to_csv('', index=False)
print("🚀 AutoGluon 최적 모델 예측 완료! 제출 파일 생성됨: submission_autogluon.csv")

🚀 AutoGluon 최적 모델 예측 완료! 제출 파일 생성됨: submission_autogluon.csv


In [ ]:
# 1️⃣ 제출 파일 불러오기
submission = pd.read_csv('')

# 2️⃣ 기존에 있던 "probability" 컬럼 제거 (만약 존재한다면)
if 'probability' in submission.columns:
    submission.drop(columns=['probability'], inplace=True)

# 3️⃣ 컬럼명 변경: "임신 성공 여부" → "probability"
submission.rename(columns={'임신 성공 여부': 'probability'}, inplace=True)

# 4️⃣ 수정된 파일 저장
submission.to_csv('', index=False)

print("🚀 최종 제출 파일 생성 완료: submission_autogluon_final.csv")

# 5️⃣ Colab에서 다운로드 가능
from google.colab import files
files.download('')

In [ ]:
binary_predictions = (y_test_pred >= 0.5).astype(int)
print(binary_predictions.sum())

4670


In [ ]:
pip install optuna


In [ ]:
pip install autogluon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. 

In [ ]:
pip install scikit-learn


In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 팝업허용 및 로그인

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
processed_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90067 entries, 0 to 90066
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   시술 유형              90067 non-null  int8    
 1   배란 자극 여부           90067 non-null  int8    
 2   단일 배아 이식 여부        87891 non-null  float64 
 3   남성 주 불임 원인         90067 non-null  int8    
 4   남성 부 불임 원인         90067 non-null  int8    
 5   여성 주 불임 원인         90067 non-null  int8    
 6   여성 부 불임 원인         90067 non-null  int8    
 7   부부 주 불임 원인         90067 non-null  int8    
 8   불명확 불임 원인          90067 non-null  int8    
 9   불임 원인 - 난관 질환      90067 non-null  int8    
 10  불임 원인 - 남성 요인      90067 non-null  int8    
 11  불임 원인 - 배란 장애      90067 non-null  int8    
 12  불임 원인 - 자궁내막증      90067 non-null  int8    
 13  총 생성 배아 수          87891 non-null  float64 
 14  미세주입된 난자 수         87891 non-null  float64 
 15  미세주입에서 생성된 배아 수    87891 non-null  float64 
 16  이식된 

In [ ]:
!pip install --upgrade lightgbm



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 32.9 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [ ]:
import lightgbm as lgb
print(lgb.__version__)


4.5.0


In [ ]:
!pip install --upgrade lightgbm --no-cache-dir


In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# 1️⃣ 데이터 로드
train = pd.read_csv('')
test = pd.read_csv('')

# 2️⃣ 타겟 컬럼 설정 및 특징/타겟 분리
target = '임신 성공 여부'
X = train.drop(columns=[target])
y = train[target]

# 3️⃣ 학습 & 검증 데이터 분할 (80% 학습, 20% 검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 변환할 컬럼 목록
bad_columns = ['시술 시기 코드', '시술 당시 나이', '시술 유형', 'IVF 임신 횟수',
               'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', '난자 출처',
               '정자 출처', '난자 기증자 나이', '정자 기증자 나이',
               '배아 이식 경과일', '시술 연속성']

# 학습 데이터와 검증 데이터의 해당 컬럼을 범주형으로 변환
for col in bad_columns:
    X_train[col] = X_train[col].astype('category')
    X_val[col] = X_val[col].astype('category')


# 4️⃣ Optuna Objective 함수 정의 (LightGBM 하이퍼파라미터 튜닝)
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'n_estimators': 1000,  # 조기 종료를 위해 크게 설정
        'random_state': 42
    }
    model = lgb.LGBMClassifier(**param)
    model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(50)]
)

    y_pred_proba = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_pred_proba)
    return auc_score

# 5️⃣ Optuna를 통한 하이퍼파라미터 최적화 (n_trials는 상황에 따라 조정)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best trial:")
best_trial = study.best_trial
print("  AUC: {:.4f}".format(best_trial.value))
print("  Best Parameters:")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))




[I 2025-02-23 01:46:00,319] A new study created in memory with name: no-name-afd3e810-001c-409b-b813-6bcddb222438


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.736915


[I 2025-02-23 01:46:58,767] Trial 0 finished with value: 0.736915355737874 and parameters: {'num_leaves': 43, 'max_depth': 7, 'learning_rate': 0.007052526219051578, 'min_child_samples': 33, 'subsample': 0.5071812214346069, 'colsample_bytree': 0.7848920906354371, 'reg_alpha': 5.894662818775544e-07, 'reg_lambda': 2.142612224754731e-08}. Best is trial 0 with value: 0.736915355737874.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's auc: 0.736607


[I 2025-02-23 01:48:06,079] Trial 1 finished with value: 0.7366068895248097 and parameters: {'num_leaves': 60, 'max_depth': 12, 'learning_rate': 0.0032572614489326124, 'min_child_samples': 93, 'subsample': 0.7399151645734068, 'colsample_bytree': 0.7770245466730308, 'reg_alpha': 0.0008126927485076498, 'reg_lambda': 7.538120879277646e-08}. Best is trial 0 with value: 0.736915355737874.


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[47]	valid_0's auc: 0.731276


[I 2025-02-23 02:12:08,119] Trial 48 finished with value: 0.731275712921609 and parameters: {'num_leaves': 30, 'max_depth': 10, 'learning_rate': 0.002548553076049082, 'min_child_samples': 71, 'subsample': 0.8803626141460202, 'colsample_bytree': 0.5372977017487901, 'reg_alpha': 0.00024750383237826104, 'reg_lambda': 2.0505496845054537e-06}. Best is trial 22 with value: 0.7374266663609165.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[711]	valid_0's auc: 0.737069


[I 2025-02-23 02:12:42,336] Trial 49 finished with value: 0.7370692066566584 and parameters: {'num_leaves': 25, 'max_depth': 5, 'learning_rate': 0.017288229286515255, 'min_child_samples': 85, 'subsample': 0.8408610038870797, 'colsample_bytree': 0.8655249022493532, 'reg_alpha': 3.735126863128714e-07, 'reg_lambda': 0.08557085951988354}. Best is trial 22 with value: 0.7374266663609165.


Best trial:
  AUC: 0.7374
  Best Parameters:
    num_leaves: 44
    max_depth: 10
    learning_rate: 0.024872783485083465
    min_child_samples: 90
    subsample: 0.8712250273596444
    colsample_bytree: 0.5038568650513761
    reg_alpha: 0.00018591024122716011
    reg_lambda: 0.17670316730852656


In [ ]:
# 6️⃣ 최적 파라미터로 최종 모델 학습
best_params = best_trial.params
# 추가로 고정 파라미터 설정
best_params.update({
    'objective': 'binary',
    'metric': 'auc',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'random_state': 42
})

final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(50)]
)


# 7️⃣ 검증 데이터 평가
y_val_pred_proba = final_model.predict_proba(X_val)[:, 1]
y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
auc = roc_auc_score(y_val, y_val_pred_proba)

print("Validation Metrics:")
print(f"🔹 Accuracy: {accuracy:.4f}")
print(f"🔹 F1-score: {f1:.4f}")
print(f"🔹 AUC: {auc:.4f}")

# 테스트 데이터 전처리: 학습 데이터와 동일한 카테고리로 설정
for col in bad_columns:
    if col in test.columns:
        test[col] = test[col].astype('category')
        test[col] = test[col].cat.set_categories(X_train[col].cat.categories)


# 8️⃣ 테스트 데이터 예측
# 테스트 데이터는 타겟 컬럼이 없으므로, 바로 예측 진행
y_test_pred_proba = final_model.predict_proba(test)[:, 1]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[336]	valid_0's auc: 0.737427
Validation Metrics:
🔹 Accuracy: 0.7459
🔹 F1-score: 0.1800
🔹 AUC: 0.7374


In [ ]:
# 9️⃣ 제출 파일 생성
submission = pd.read_csv('/')
submission[target] = y_test_pred_proba
submission.to_csv('', index=False)

print("🚀 LightGBM 최적 모델 예측 완료! 제출 파일 생성됨: submission_lightgbm.csv")

# 10️⃣ Colab에서 제출 파일 다운로드
from google.colab import files
files.download('')


🚀 LightGBM 최적 모델 예측 완료! 제출 파일 생성됨: submission_lightgbm.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 1️⃣ 제출 파일 불러오기
submission = pd.read_csv('')

# 2️⃣ 기존에 있던 "probability" 컬럼 제거 (만약 존재한다면)
if 'probability' in submission.columns:
    submission.drop(columns=['probability'], inplace=True)

# 3️⃣ 컬럼명 변경: "임신 성공 여부" → "probability"
submission.rename(columns={'임신 성공 여부': 'probability'}, inplace=True)

# 4️⃣ 수정된 파일 저장
submission.to_csv('', index=False)

print("🚀 최종 제출 파일 생성 완료: submission_lightgbm_final.csv")

# 5️⃣ Colab에서 다운로드 가능
from google.colab import files
files.download('')

🚀 최종 제출 파일 생성 완료: submission_lightgbm_final.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
binary_predictions = (y_test_pred >= 0.5).astype(int)
print(binary_predictions.sum())

4749
